In [27]:
import pandas as pd
import numpy as np
import sklearn as sk
import math

In [28]:
dist = pd.read_csv('dist_data.csv', sep=';')

In [29]:
dist_clean = dist[dist['dist'] != "[[], []]"]

In [30]:
x = dist_clean.iloc[:, :-1].values
y = dist_clean.iloc[:, 1].values

In [31]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [32]:
def clean_sample(sample):
    sample_list = sample[0].replace(', [', '; [').split(';')
    sample_x = sample_list[0]
    sample_Px = sample_list[1]

    sample_x_list = []
    sample_Px_list = []
    
    for x in sample_x.split(','):
        sample_x_list.append(x.replace(' ', '').replace('[', '').replace(']', ''))
    for Px in sample_Px.split(','):
        sample_Px_list.append(Px.replace(' ', '').replace('[', '').replace(']', ''))

    dict_sample = {}
    for i in range(len(sample_x_list)):
        dict_sample[sample_x_list[i]] = sample_Px_list[i]
    return dict_sample

In [33]:
def all_keys_dataset1_exist_in_dataset2(dataset1, dataset2, threshold):
    count = 0
    total = len(dataset1.keys())
    for key in dataset1.keys():
        if dataset2.get(key) is None:
            pass
        else:
            count += 1

    if (count/total) * 100 >= float(threshold):
        #print(count, (count/total) * 100, float(threshold))
        return True
    else:
        return False

In [34]:
def calculate_distance(distance_func, min_precent_similarity):
    # count = 0
    result = []
    for i, sample_x_test in enumerate(x_test): 
        result_zikma = []
    
        for j, sample_x in enumerate(x):
            if all_keys_dataset1_exist_in_dataset2(clean_sample(sample_x_test), clean_sample(sample_x), min_precent_similarity):
                Px_list = []
                for xd1, Pxd1 in clean_sample(sample_x_test).items():
                    if xd1 == "" or Pxd1 == "":
                        continue
                    Pxd2 = clean_sample(sample_x).get(xd1)
                    if Pxd2 is None:
                        Pxd2 = 0.0
                    Px_list.append([Pxd1, Pxd2])
                    
                for xd2, Pxd2 in clean_sample(sample_x).items():
                    if xd2 == "" or Pxd2 == "":
                        continue
                    Pxd1 = clean_sample(sample_x_test).get(xd2)
                    if Pxd1 is None:
                        Px_list.append([0.0, Pxd2])
                
                sum = distance_func(Px_list)
                result_zikma.append([y[j], sum])
                #print(Px_list, sum)
                
            else:
                continue
        
        result.append([sample_x_test ,result_zikma])
        
        # count += 1
        # if count > 10:
        #     break
    
    return result

In [35]:
def js(Px_list):
    for i in Px_list:
        Mx = (float(i[0]) + float(i[1]))/2
        i.append(Mx)

    PM_list = []
    QM_list = []
    for i in Px_list:
        PM_list.append([i[0], i[2]])
        QM_list.append([i[1], i[2]])

    sum1 = kl(PM_list)
    sum2 = kl(QM_list)

    return (sum1 + sum2)/2

In [36]:
def kl(Px_list):
    sum = 0 
    for i in Px_list:
        Pxd1 = i[0]
        Pxd2 = i[1]
        if float(Pxd1) == 0.0:
            sum += 0.0
            continue
        sum += float(Pxd1) * math.log10(float(Pxd1) / float(Pxd2))
    return sum

In [37]:
def Predict(all_distance_label, threshold):
    y_pred = []
    for i in range(len(x_test)):
        t = pd.DataFrame(all_distance_label[i][1], columns=['Label', 'Kl Result'])
        tmp = t[t['Kl Result'] < threshold] #0.99 - 1.01
        grouped_df = tmp.groupby(['Label']).size().reset_index(name='Count')
        sorted_df = grouped_df.sort_values(by=['Count'], ascending=False)
        
        if tmp.values.size == 0:
            y_pred.append(0)
        else:
            y_pred.append(sorted_df.head(1)['Label'].values[0])
    return y_pred

In [38]:
def diffrence_pred_test(y_pred, y_test):
    test = []
    count = 0
    for i, value1 in enumerate(y_pred):
        if value1 != y_test[i]:
            test.append([i, value1, y_test[i]])
            count+=1
    
    return pd.DataFrame(test, columns=['index', 'y_pred', 'y_test'])

In [39]:
final_result_js_full = calculate_distance(js, 100)

In [40]:
y_pred_js = Predict(final_result_js_full, 0.093) #good threshold 0.093

In [41]:
diffrence_pred_test(y_pred_js, y_test)

,index,y_pred,y_test
0,9,27525,27341
1,14,27525,27341
2,23,27525,27341
3,34,27525,27523
4,40,27523,27652
...,...,...,...
119,1494,27523,27321
120,1517,27523,27321
121,1535,27525,27341
122,1549,27523,27652


In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res1 = confusion_matrix(y_test, y_pred_js)
print("Confusion Matrix:")
print(res1)
res11 = classification_report(y_test, y_pred_js)
print("Classification Report:",)
print (res11)
res12 = accuracy_score(y_test,y_pred_js)
print("Accuracy:",res12)

Confusion Matrix:
[[131   0   0   2   0   0   0]
 [  0 331   0  23  30   0   0]
 [  0   0   0   1  38   0   0]
 [  1   5   0 283  14   0   0]
 [  0   1   0   1 594   0   0]
 [  0   0   0   8   0  17   0]
 [  0   0   0   0   0   0 102]]
Classification Report:
              precision    recall  f1-score   support

       27311       0.99      0.98      0.99       133
       27321       0.98      0.86      0.92       384
       27341       0.00      0.00      0.00        39
       27523       0.89      0.93      0.91       303
       27525       0.88      1.00      0.93       596
       27652       1.00      0.68      0.81        25
       28090       1.00      1.00      1.00       102

    accuracy                           0.92      1582
   macro avg       0.82      0.78      0.79      1582
weighted avg       0.90      0.92      0.91      1582

Accuracy: 0.9216182048040455


/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,